# Entropy of each frame in a video

In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
from PIL import Image

### Hotspot Region

In [ ]:
def crop_image(img: np.ndarray, hot_spot: dict[str, int]) -> np.ndarray:
    """
    Crops an image based on given coordinates from a hotspot dictionary.
    """
    if img is None or img.shape[0] == 0 or img.shape[1] == 0:
        raise ValueError("Invalid image: Image is empty or has incorrect dimensions.")

    required_keys = {"top", "left", "bottom", "right"}
    missing_keys = required_keys - set(hot_spot.keys())
    if missing_keys:
        raise ValueError(
            f"Invalid hotspot dictionary: Missing required keys: {sorted(missing_keys)}"
        )

    if hot_spot["left"] >= hot_spot["right"] or hot_spot["top"] >= hot_spot["bottom"]:
        raise ValueError("Starting coordinates must be less than ending coordinates.")

    x1 = max(0, hot_spot["left"])
    y1 = max(0, hot_spot["top"])
    x2 = min(img.shape[1], hot_spot["right"])
    y2 = min(img.shape[0], hot_spot["bottom"])

    return img[y1:y2, x1:x2]


def get_entropy(frame: np.ndarray, hot_spot: dict) -> tuple[float, np.ndarray]:
    """
    Calculates the entropy of a cropped region from an image.
    Returns (entropy, cropped_region_rgb).
    """
    hot_spot_img = crop_image(frame, hot_spot)
    # BGR -> RGB for PIL
    hot_spot_img = Image.fromarray(cv2.cvtColor(hot_spot_img, cv2.COLOR_BGR2RGB))

    return hot_spot_img.entropy(), cv2.cvtColor(
        np.array(hot_spot_img), cv2.COLOR_BGR2RGB
    )


def process_images_in_folder(image_folder, hotspot, entropy_value):
    image_folder = Path(image_folder)
    entropy_dict = {}

    for image_file in image_folder.glob("*.jpg"):
        image = cv2.imread(str(image_file))
        if image is None:
            print(f"Failed to load image: {image_file}")
            continue

        entropy, hot_spot_img = get_entropy(image, hotspot)
        entropy_dict[image_file] = entropy
        print(f"Processing {image_file}, Entropy: {entropy}")

        if entropy < entropy_value:
            os.remove(str(image_file))

    entropy_list = list(entropy_dict.values())
    return entropy_list, entropy_dict


def process_video_with_entropy_overlay(
    input_video_path: str,
    output_video_path: str,
    hotspot: dict[str, int],
    font_scale: float = 1.0,
    thickness: int = 2,
):
    """
    Reads a video, computes entropy for each frame using the hotspot, overlays
    the entropy value as text on the frame, and writes a new video.

    Args:
        input_video_path: Path to the input video.
        output_video_path: Path to the output video to be created.
        hotspot: Dict with keys 'top', 'bottom', 'left', 'right' for cropping.
        font_scale: OpenCV font scale for the text.
        thickness: Thickness of the text stroke.

    Returns:
        List of entropy values, one per frame.
    """
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_video_path}")

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # or 'XVID', depending on your needs
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    entropy_values: list[float] = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        try:
            entropy, _ = get_entropy(frame, hotspot)
        except ValueError as e:
            print(f"Frame {frame_idx}: error during entropy calculation: {e}")
            # Still write original frame (no entropy text) if desired
            out.write(frame)
            frame_idx += 1
            continue

        entropy_values.append(entropy)

        # Prepare text to overlay
        text = f"Entropy: {entropy:.4f}"

        # Choose position and font
        org = (20, 40)  # (x, y)
        font = cv2.FONT_HERSHEY_SIMPLEX

        # Optional: put a filled rectangle behind text to improve readability
        (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)
        cv2.rectangle(
            frame,
            (org[0] - 5, org[1] - text_h - 5),
            (org[0] + text_w + 5, org[1] + baseline + 5),
            (0, 0, 0),  # black background
            thickness=cv2.FILLED,
        )

        # Put text on frame (white text)
        cv2.putText(
            frame,
            text,
            org,
            font,
            font_scale,
            (255, 255, 255),
            thickness,
            cv2.LINE_AA,
        )

        # Write the frame with text to output video
        out.write(frame)

        print(f"Frame {frame_idx}: {text}")
        frame_idx += 1

    cap.release()
    out.release()

    print(f"Saved video with entropy overlay to: {output_video_path}")
    return entropy_values

In [ ]:
hotspot = {
    "top": 100,
    "bottom": 300,
    "left": 200,
    "right": 400,
}

input_video = "../videos/100_crop1.avi"
output_video = "../videos/100_crop1_entropy.avi"

entropies = process_video_with_entropy_overlay(
    input_video_path=input_video,
    output_video_path=output_video,
    hotspot=hotspot,
    font_scale=0.8,
    thickness=2,
)

###  ENTIRE FRAME

In [ ]:
def entropy_from_channel(channel):
    """
    Compute Shannon entropy from a single image channel.
    """
    # Flatten to 1D
    pixels = channel.flatten()

    # Compute histogram with 256 bins
    hist, _ = np.histogram(pixels, bins=256, range=(0, 256))

    # Convert to probability distribution
    p = hist / np.sum(hist)

    # Remove zeros
    p = p[p > 0]

    # Shannon entropy
    return -np.sum(p * np.log2(p))

def get_entropy_full_frame(frame: np.ndarray) -> float:
    """
    Calculates entropy of the entire frame.

    Args:
        frame: The input image in BGR color space (as read by OpenCV).

    Returns:
        Entropy (float)
    """
    # Convert BGR -> RGB for PIL
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # R, G, B = cv2.split(img_rgb)
    # entropy_R = entropy_from_channel(R)
    # return entropy_R
    
    pil_img = Image.fromarray(img_rgb)
    return pil_img.entropy()

def process_video_with_entropy_overlay_fullframe(
    input_video_path: str,
    output_video_path: str,
    font_scale: float = 1.0,
    thickness: int = 2,
):
    """
    Calculates entropy on the FULL FRAME (no hotspot),
    overlays the entropy value as text, and writes a new video.
    """

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_video_path}")

    # Get video info
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    entropy_values = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # --- ENTROPY OF ENTIRE FRAME ---
        entropy = get_entropy_full_frame(frame)
        entropy_values.append(entropy)

        text = f"Entropy: {entropy:.4f}"
        org = (20, 40)
        font = cv2.FONT_HERSHEY_SIMPLEX

        # Draw readable box behind text
        (tw, th), baseline = cv2.getTextSize(text, font, font_scale, thickness)
        cv2.rectangle(
            frame,
            (org[0] - 5, org[1] - th - 5),
            (org[0] + tw + 5, org[1] + baseline + 5),
            (0, 0, 0),
            thickness=cv2.FILLED,
        )

        # Draw text
        cv2.putText(
            frame,
            text,
            org,
            font,
            font_scale,
            (255, 255, 255),
            thickness,
            cv2.LINE_AA,
        )

        out.write(frame)
        print(f"Frame {frame_idx}: Entropy = {entropy:.4f}")
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Saved entropy video to {output_video_path}")

    return entropy_values


In [ ]:
input_video = "../videos/100_crop1.avi"
output_video = "../videos/100_crop1_entropy.avi"

entropies = process_video_with_entropy_overlay_fullframe(
    input_video_path=input_video,
    output_video_path=output_video,
    font_scale=0.8,
    thickness=2,
)